In [ ]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import skimage
import cv2

In [ ]:
%matplotlib inline

In [ ]:
path = '/scratch/ssd/cciw/sample_data/'
#img = '1340_2016-07-11_1_GLN_3447.JPG'
img = '1349_2016-07-06_2_GLN_3061.JPG'
#img = 'sudoku.png'

In [ ]:
f = os.path.join(path, img)

In [ ]:
im = cv2.imread(f)

In [ ]:
im.shape

In [ ]:
#np.sqrt(im.shape[0]**2 + im.shape[1]**2)

In [ ]:
ds = 8
plt.imshow(im[::ds, ::ds])

In [ ]:
#img = im[::ds, ::ds].astype(np.uint8).copy()
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
edges = cv2.Canny(gray, 100, 500)
plt.imshow(edges)

In [ ]:
'''
@param rho Distance resolution of the accumulator in pixels.
@param theta Angle resolution of the accumulator in radians.
@param threshold Accumulator threshold parameter. Only those lines are returned that get enough
       votes ( \f$>\texttt{threshold}\f$ ).
@param minLineLength Minimum line length. Line segments shorter than that are rejected.
@param maxLineGap Maximum allowed gap between points on the same line to link them.
'''
#img = im[::ds, ::ds].astype(np.uint8).copy()
img = np.ascontiguousarray(im[::ds, ::ds], dtype=np.uint8)
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
edges = cv2.Canny(gray, 100, 500)
#plt.imshow(edges[:, 200:])
theta = np.pi / 90  # note, with 180 getting too many overlapping lines.
lines = cv2.HoughLines(edges, 1, theta, 10)
print(lines.shape)
#coords = []
N = 2
coords = np.zeros((N, 2, 2)) # points, start/end, x/y
for i in range(len(lines[:N])):
    for rho, theta in lines[i]:
        print(i, abs(theta * 180 / np.pi - 90))
        #if i == 0:
        #    t = theta
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a*rho
        y0 = b*rho
        x1 = int(x0 + img.shape[1]*(-b))
        y1 = int(y0 + img.shape[1]*(a))
        x2 = int(x0 - img.shape[1]*(-b))
        y2 = int(y0 - img.shape[1]*(a))
        #coords.append([[x1, y1], [x2, y2]])
        coords[i, 0, 0] = x1
        coords[i, 0, 1] = y1
        coords[i, 1, 0] = x2
        coords[i, 1, 1] = y2
        print(i, x1, y1, x2, y2)
        cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
plt.imshow(img)        
plt.xlabel('y')
plt.ylabel('x')

In [ ]:
%matplotlib

In [ ]:
theta = lines[0, 0, 1]

In [ ]:
theta

In [ ]:
t = abs(theta - np.pi / 2)
R = np.array([[np.cos(t), -np.sin(t)],
              [np.sin(t),  np.cos(t)]])

In [ ]:
print(coords)
coords_r = np.dot(coords, R)
print(coords_r)

In [ ]:
print(coords_r[0, 0, 0]) 
print(coords_r[0, 1, 0])
idx = int((coords_r[0, 0, 0] + coords_r[0, 1, 0]) / 2)
print(idx)

#print(coords_r[2, 0, 0]) 
#print(coords_r[2, 1, 0])
#idx = int((coords_r[2, 0, 0] + coords_r[2, 1, 0]) / 2)
#print(idx)

In [ ]:
imgr = skimage.transform.rotate(img, abs(theta - np.pi/2)*180/np.pi)#.astype(np.uint8)
imgr = (255 * imgr).astype(np.uint8)
plt.imshow(imgr[:, :])

In [ ]:
#np.dot()
'''
data = np.random.multivariate_normal(mean=[0, 0], 
                                     cov=np.array([[0.1, 0], [0, 0.001]]), size=1000)

plt.scatter(data[:, 0], data[:, 1])
dr = np.dot(data, R)
plt.scatter(dr[:, 0], dr[:, 1])
plt.xlim(-3, 3)
plt.ylim(-3, 3)
'''

In [ ]:
x0, y0

In [ ]:
run = coords[0][2] - coords[0][0]
rise = coords[0][3] - coords[0][1]
print(rise)
print(run)
print(rise/run)

In [ ]:
#imgr.max()

In [ ]:
#imgr_c.dtype

In [ ]:
imgr_c = img.copy()
gray = cv2.cvtColor(imgr_c, cv2.COLOR_RGB2GRAY)
edges = cv2.Canny(gray, 100, 500)

In [ ]:
plt.imshow(edges)

In [ ]:
lines = cv2.HoughLinesP(
    edges, rho=1, theta=np.pi/90, threshold=10, minLineLength=300, maxLineGap=70)
print(lines.shape)
hp_coords = np.zeros((N, 2, 2)) # points, start/end, x/y
for i in range(len(lines[:N])):
    for x1,y1,x2,y2 in lines[i]:
        print(x1, y1, x2, y2)
        hp_coords[i, 0, 0] = x1
        hp_coords[i, 0, 1] = y1
        hp_coords[i, 1, 0] = x2
        hp_coords[i, 1, 1] = y2
        cv2.line(imgr_c,(x1,y1),(x2,y2),(0,0,255),2)

In [ ]:
#imgr = skimage.transform.rotate(img, theta * 180/np.pi)
#plt.imshow(imgr[40:380, 330:660])
plt.imshow(imgr_c)